# MTH3302: Méthodes probabilistes et statistiques pour l'I.A.
## Projet

- Vincent Haney
- Félix Pelletier
- Charles-François St-Cyr (275354)
- Mikael Vaillant (2029994)


### Librairies

In [2]:
import Pkg;
packages = ["CSV", "DataFrames", "Distributions", "Gadfly", "MLBase", "Random", "Statistics", "Colors", "GLM", "LinearAlgebra", "Combinatorics", "CategoricalArrays","Distances","NearestNeighbors"];
# peut aps être mis dans une fonction. using doit être top-level
for package in packages
    try
        @eval using $(Symbol(package))
    catch e
        Pkg.add(package)
        @eval using $(Symbol(package))
    end
end
Pkg.update()

include("./library/main.jl")
include("./library/normalize.jl")

    Updating registry at `C:\Users\vaill\.julia\registries\General.toml`
  No Changes to `C:\Users\vaill\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\vaill\.julia\environments\v1.8\Manifest.toml`


normalize (generic function with 9 methods)

### Chargement des données (partiel pour l'exploration)

In [3]:
data = CSV.read("./data/train.csv", DataFrame)
test = CSV.read("./data/test.csv", DataFrame)

# data[!, :volume] .= data.x .* data.y .* data.z

train, valid = partitionData(data, 0.75);

first(train, 5)

Row,ID,cut,color,clarity,depth,table,price,x,y,z
,Int64,String15?,String1,String7,Float64?,Float64,Int64,Float64,Float64?,Float64
1,1,missing,E,SI2,60.2,60.0,5340,6.46,6.5,3.9
2,4,Ideal,G,VS2,61.7,57.0,720,4.41,4.38,2.71
3,5,missing,G,VS2,62.4,58.0,6301,6.41,6.48,4.02
4,6,Premium,D,VVS2,61.5,60.0,925,4.36,4.42,2.7
5,7,Very Good,D,SI1,60.2,61.0,571,4.39,4.42,2.65


### Visualisation des données

In [4]:
# plot(data, x = :price, Geom.histogram())

In [5]:
missing_counts = collect(count(ismissing,col) for col in eachcol(data))
println("Nombre de valeurs manquantes dans les différentes colonnes: ")
println(missing_counts)

zero_counts = [count(x -> x == 0, skipmissing(data[:, col])) for col in names(data)]
println("Nombre de valeurs nulles dans les différentes colonnes: ")
println(zero_counts)


Nombre de valeurs manquantes dans les différentes colonnes: 
[0, 3806, 0, 0, 2201, 0, 0, 0, 2200, 0]
Nombre de valeurs nulles dans les différentes colonnes: 
[0, 0, 0, 0, 13, 0, 0, 2, 1, 15]


In [4]:
function fillDataFromAverage(data::DataFrame; is_test::Bool = false)
    avgCut = mode(dropmissing(data, :cut).cut)
    avgColor = mode(dropmissing(data, :color).color)
    avgClarity = mode(dropmissing(data, :clarity).clarity)
    avgDepth = mean(dropmissing(data, :depth).depth)
    avgTable = mean(dropmissing(data, :table).table)
    avgX = mean(dropmissing(data, :x).x)
    avgY = mean(dropmissing(data, :y).y)
    avgZ = mean(dropmissing(data, :z).z)
    # avgVol = mean(dropmissing(data, :volume).volume)

    replace!(data.cut, missing => avgCut)
    replace!(data.color, missing => avgColor)
    replace!(data.clarity, missing => avgClarity)
    replace!(data.depth, missing => avgDepth)
    replace!(data.table, missing => avgTable)
    replace!(data.x, missing => avgX)
    replace!(data.y, missing => avgY)
    replace!(data.z, missing => avgZ)
    # replace!(data.volume, missing => avgVol)

    if (!is_test)
        avgPrice = mean(dropmissing(data, :price).price)
        replace!(data.price, missing => avgPrice)
    end
end

fillDataFromAverage (generic function with 1 method)

In [18]:
train₁ = deepcopy(train)
valid₁ = deepcopy(valid)
test₁ = deepcopy(test)

fillDataFromAverage(train₁)
fillDataFromAverage(valid₁)
fillDataFromAverage(test₁, is_test = true)

convertQualitativeToQuantitative(train₁, :color)
# convertQualitativeToQuantitative(train₁, :cut)
convertQualitativeToQuantitative(train₁, :clarity)

convertQualitativeToQuantitative(valid₁, :color)
# convertQualitativeToQuantitative(valid₁, :cut)
convertQualitativeToQuantitative(valid₁, :clarity)

convertQualitativeToQuantitative(test₁, :color)
# convertQualitativeToQuantitative(test₁, :cut)
convertQualitativeToQuantitative(test₁, :clarity)

price_train = select(train₁, (:price))
price_true = select(valid₁, (:price))

train₁ = select(train₁, Not(["price","cut","ID","color","clarity","y","depth"]))
valid₁ = select(valid₁, Not(["price","cut","ID","color","clarity","y","depth"]))

for col in names(train₁[:,1:5])
    train₁[!, col] = (train₁[!, col] .- mean(train₁[!, col])) ./ std(train₁[!, col])
end

for col in names(valid₁[:,1:5])
    valid₁[!, col] = (valid₁[!, col] .- mean(valid₁[!, col])) ./ std(valid₁[!, col])
end



first(train₁,5)

Row,table,x,z,color: E,color: F,color: G,color: H,color: I,color: J,clarity: IF,clarity: SI1,clarity: SI2,clarity: VS1,clarity: VS2,clarity: VVS1,clarity: VVS2
,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1.13315,0.644654,0.515256,2.12353,-0.463517,0,0,0,0,0,0,1,0,0,0,0
2,-0.205803,-1.18342,-1.19514,-0.470898,-0.463517,1,0,0,0,0,0,0,0,1,0,0
3,0.240513,0.600067,0.687734,-0.470898,-0.463517,1,0,0,0,0,0,0,0,1,0,0
4,1.13315,-1.228,-1.20952,-0.470898,-0.463517,0,0,0,0,0,0,0,0,0,0,1
5,1.57946,-1.20125,-1.28138,-0.470898,-0.463517,0,0,0,0,0,1,0,0,0,0,0


#### Prédiction

In [19]:

data = Matrix(train₁)'
k = 4
point = Matrix(valid₁)'

kdtree = KDTree(data)
idxs, dists = knn(kdtree, point, k, true)

prediction = zeros(size(idxs))
for i in 1:size(idxs,1)
    prediction[i] =  mean(Matrix(price_train)[idxs[i]])
end

RMSE₁₁ = sqrt(mean((prediction .- Matrix(price_true)).^2))



737.3883193904769